# Montana Board of Oil and Gas Conservation

The purpose of this notebook is to outline the process for collecting oil and gas data from the state of Montana, how to transform the data into something useful for analysis, and to perform some initial EDA

# Data Collection
Start by extracting the data from the following link

In [1]:
# importing libraries
import zipfile
from urllib.request import urlopen
import shutil
import os
import pandas as pd

url = 'http://bogc.dnrc.mt.gov/production/historical.zip'
file_name = 'historical.zip'

# extracting zipfile from URL
with urlopen(url) as response, open(file_name, 'wb') as out_file:
    shutil.copyfileobj(response, out_file)

    # extracting required file from zipfile
    with zipfile.ZipFile(file_name) as zf:
        zf.extract('histLeaseProd.tab')
        zf.extract('histprodwell.tab')
        zf.extract('histWellData.tab')

# deleting the zipfile from the directory
os.remove('historical.zip')

# loading data from the file
lease_prod_df = pd.read_csv('histLeaseProd.tab', sep='\t')
well_prod_df = pd.read_csv('histprodwell.tab', sep='\t')
well_data_df = pd.read_csv('histWellData.tab', sep='\t')

C:\Users\josep\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3172: DtypeWarning: Columns (12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


# View Data
Let's start by taking a look at these 3 files pulled from the .zip file. There are three flat files. `lease_prod_df` contains production reported on a lease basis, `well_prod_df` contains production on a per well monthly basis, and `well_data_df` contains the well information (Producting status, field, horizontal, vertical drill profile, etc.)

In [2]:
lease_prod_df.head()

,Lease_Unit,Rpt_Date,Dt_Receive,Del_Rpt,Amnd_Rpt,OpNo,CoName,StartIvn_OilCd,Oil_Prod,Gas_Prod,...,WtrInj,WtrTo_Pit,Other_Oil,Other_Gas,Other_Wtr,Dt_Amend,Lease_Update,No_ProdWells,No_SIWells,Dt_Mod
0,2,01/31/2001,03/15/2001,False,False,5385,EnCana Oil & Gas (USA) Inc.,0.0,0.0,353.0,...,0.0,0.0,0.0,0.0,0.0,NaN,False,1.0,0.0,05/18/2001
1,3,01/31/2001,03/15/2001,False,False,5385,EnCana Oil & Gas (USA) Inc.,0.0,0.0,69.0,...,0.0,0.0,0.0,0.0,0.0,NaN,False,1.0,0.0,05/18/2001
2,4,01/31/2001,03/05/2001,False,False,6681,Samedan Oil Corporation,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,NaN,False,0.0,1.0,03/06/2001
3,5,01/31/2001,03/05/2001,False,False,6681,Samedan Oil Corporation,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,NaN,False,0.0,1.0,03/06/2001
4,6,01/31/2001,03/05/2001,False,False,6681,Samedan Oil Corporation,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,NaN,False,0.0,1.0,03/06/2001


In [5]:
lease_prod_df.describe()

,Lease_Unit,OpNo,StartIvn_OilCd,Oil_Prod,Gas_Prod,Wtr_Prod,Oil_Sold,Gas_Sold,OilSpill,WtrSpill,...,UseGas,OilInj,GasInj,WtrInj,WtrTo_Pit,Other_Oil,Other_Gas,Other_Wtr,No_ProdWells,No_SIWells
count,1.298191e+06,1.298191e+06,1.283575e+06,1.298181e+06,1.297562e+06,1.297373e+06,1.297370e+06,1.298186e+06,1.297238e+06,78134.000000,...,1.298188e+06,1.297233e+06,1.298187e+06,1.279484e+06,1.279527e+06,1.286605e+06,1.298187e+06,1.279363e+06,1.298170e+06,1.288983e+06
mean,1.370504e+04,1.717545e+03,9.998504e+01,4.047499e+02,1.476859e+03,2.764494e+03,4.047213e+02,1.132646e+03,3.033368e-03,0.020900,...,4.778551e+01,3.475860e-03,2.352886e+02,1.616128e+03,3.201118e+02,1.094641e-01,3.267775e+00,2.214530e+02,1.815966e+00,7.251414e-01
std,3.718679e+04,2.344972e+03,2.078206e+02,3.377169e+03,3.272934e+04,3.604075e+04,3.378501e+03,9.589917e+03,6.614214e-01,5.485964,...,4.437782e+02,1.120417e+00,3.114522e+04,3.160498e+04,9.859811e+03,1.468871e+01,1.465689e+02,8.156987e+03,1.024098e+01,4.598063e+00
min,2.000000e+00,4.000000e+00,-8.150000e+02,-4.020000e+02,0.000000e+00,-5.150000e+02,-2.300000e+02,0.000000e+00,0.000000e+00,0.000000,...,-8.190000e+02,-4.800000e+01,0.000000e+00,-9.300000e+01,-5.000000e+01,-8.580000e+03,-5.540000e+02,-7.055000e+04,0.000000e+00,0.000000e+00
25%,2.666000e+03,3.210000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00
50%,5.251000e+03,5.360000e+02,0.000000e+00,0.000000e+00,1.380000e+02,0.000000e+00,0.000000e+00,6.900000e+01,0.000000e+00,0.000000,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00
75%,7.387000e+03,1.720000e+03,1.460000e+02,1.170000e+02,7.100000e+02,1.250000e+02,1.220000e+02,5.980000e+02,0.000000e+00,0.000000,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00
max,9.900150e+05,1.230600e+04,8.493000e+03,2.318970e+05,5.934068e+06,6.684198e+06,2.319860e+05,6.788670e+05,2.480000e+02,1530.000000,...,7.351400e+04,9.890000e+02,5.934068e+06,2.096867e+06,6.684198e+06,3.000000e+03,2.473100e+04,8.252006e+06,1.154000e+03,1.200000e+03


In [3]:
well_prod_df.head()

,rpt_date,API_WELLNO,ST_FMTN_CD,Name_,Lease_Unit,OPNO,CoName,BBLS_OIL_COND,MCF_GAS,BBLS_WTR,DAYS_PROD,AMND_RPT,STATUS,dt_mod
0,12/30/1899,25035061990000,CB,Cut Bank,2593.0,1770.0,Cut Bank Gas Company,0.0,0.0,0,0.0,False,NaN,04/13/2020
1,12/30/1899,25035062030000,CB,Cut Bank,2593.0,1770.0,Cut Bank Gas Company,0.0,0.0,0,0.0,False,NaN,04/13/2020
2,12/30/1899,25035062220000,CB,Cut Bank,2593.0,1770.0,Cut Bank Gas Company,0.0,0.0,0,0.0,False,NaN,04/13/2020
3,12/30/1899,25035062380000,CB,Cut Bank,2593.0,1770.0,Cut Bank Gas Company,0.0,0.0,0,0.0,False,NaN,04/13/2020
4,12/30/1899,25035062860000,CB,Cut Bank,2203.0,1770.0,Cut Bank Gas Company,0.0,0.0,0,0.0,False,NaN,04/10/2020


In [4]:
well_data_df.head()

,API_WellNo,OpNo,CoName,Well_Nm,Well_Typ,Type,Wl_Status,Status,Wh_Sec,Wh_Twpn,...,Wh_EW,Slant,Reg_Field_No,Reg_Field,Stat_Field_No,Stat_Field,Dt_APD,Dt_Cmp,Elev_KB,DTD
0,25101100140000,7120,"Somont Oil Company, Inc.",Remington-Warner 7,OIL,Oil,PR,Producing,32,35,...,W,V,4820,Kevin-Sunburst,4820,Kevin-Sunburst,NaN,1927-07-05 00:00:00,NaN,1500.0
1,25025225200000,664,"Denbury Onshore, LLC",Little Beaver East 23-22H,OIL,Oil,SI,Shut In,22,5,...,W,H,5420,"Little Beaver, East",5420,"Little Beaver, East",2005-04-12 00:00:00,2006-01-27 00:00:00,2989.0,11250.0
2,25073210450000,5130,Mont Mil Operating Company,TMCBSU 10-10,EOR,"Injection, EOR",AX,P&A - Approved,10,31,...,W,V,2400,Cut Bank,2400,Cut Bank,1968-11-25 00:00:00,1969-07-15 00:00:00,NaN,3422.0
3,25065055920000,645,Kelly Oil and Gas LLC,Smith M #3,OIL,Oil,SI,Shut In,12,10,...,E,V,4700,Keg Coulee,4700,Keg Coulee,1966-10-20 00:00:00,1966-12-17 00:00:00,NaN,4855.0
4,25101226430000,4070,Kipling Energy Incorporated,Allen 4,OIL,Oil,AX,P&A - Approved,11,34,...,E,V,4820,Kevin-Sunburst,4820,Kevin-Sunburst,1984-07-20 00:00:00,1984-08-16 00:00:00,NaN,1588.0


# Clean and Merge 

There are two ways that we can look at our data, we can choose to look through the lenses of leases which will contain a collection of 1 or more wells. The other is to perform analysis on a per well Basis